In [ ]:
import watertap.examples.flowsheets.electrodialysis.electrodialysis_1stack_concentrate_recirc as ef
import importlib
from idaes.core.solvers import get_solver
import pandas as pd
import numpy as np
from pyomo.environ import value, check_optimal_termination
import idaes.logger as idaeslog
importlib.reload(ef)

In [ ]:
# To visualize the product salinity as a funciton of applied potential at 
# different feed salinity and water recoveries. 
_log = idaeslog.getLogger(__name__)
solver = get_solver()
m = ef.build()
fd_sal=[1, 5, 15] # in g/L
c_mass_fd=pd.DataFrame(fd_sal,columns=["c0_mass"])
initarg_list=ef.make_initarg_list(fd_sal)
wr_list = [0.5, 0.7, 0.9]
wr=pd.DataFrame(wr_list,columns=["WR"])
volt_list= list(np.arange(5, 25, 5))
volt=pd.DataFrame(volt_list,columns=["Volt"])
c_wr_v=c_mass_fd.merge(wr,how='cross').merge(volt,how='cross')
solver = get_solver()
if not outdt is None:
    del outdt
outdt=c_wr_v.copy()
for i,r in c_wr_v[(c_wr_v["c0_mass"]==1) & (c_wr_v["WR"]==0.7)].iterrows():
    m.fs.feed.find_component("flow_mol_phase_comp").unfix()
    initarg=ef.make_initarg_list([r["c0_mass"]])
    deci_var_dict={"voltage_applied[0]":r["Volt"],"recovery_vol_H2O": r["WR"]}
    try:
        deci_comp=ef.initialize_dof0_system(m=m,initargs=initarg[0],**deci_var_dict)
    except:
        _log.warning("Initialization FAILING: {}".format(r))
        pass
    solres=solver.solve(m)
    if check_optimal_termination(solres):
        outdt.loc[i,"c_prod_mass"]= value(m.fs.prod_salinity)
        outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
    else:
        _log.warning("Solving FAILING at {} with solver status {}".format(r,solres.solver.status))
        outdt.loc[i,"c_prod_mass"]= float("nan")
        outdt.loc[i,"Sol_Cond"]=solres.solver.termination_condition
    for comp in deci_comp:
        comp.unfix()
 
print(outdt)   